# Red Neuronal (?) Inception V3 con Tensor.flow Keras

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os


c:\Users\juang\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\juang\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [1]:
cumulos_folder_shell1=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell1\cumulos"
nocumulos_folder_shell1=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell1\no-cumulos"

cumulos_folder_shell2=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell2\cumulos"
nocumulos_folder_shell2=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell2\no-cumulos"

shell1=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell1"
shell2=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\muestra_entrenamiento\shell2"

clasificar_shell1=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\data_v2\shell1"
clasificar_shell2=r"C:\Users\juang\Documents\Tareas JD\Universidad\Materias\8 Semestre\Proyecto Teorico\Proyecto Teorico 20242\Gaia-DR3-Proyecto-Teorico-20242\data_v2\shell2"

In [ ]:
# Paths to the datasets
train_data_dir = shell1 # Cambiar a shell1 o shell2 segun corresponda

# Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Rescale pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load training data from 'clusters' and 'not_clusters'
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(299, 299),  # InceptionV3 requires 299x299 input size
    batch_size=32,
    class_mode='binary',  # Binary classification: "clusters" vs "not_clusters"
    classes=['cumulos', 'no-cumulos']  # Map directories to labels
)

# Data preprocessing for the unknown folder (without augmentation)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

unknown_generator = test_datagen.flow_from_directory(
    clasificar_shell1, #TODO Organizar directorio con datos a clasificar
    target_size=(299, 299),
    batch_size=32,
    class_mode=None,  # No labels for unknown data
    shuffle=False  # Keep the order of files
)


In [ ]:
# Load the InceptionV3 model pre-trained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',  # Binary classification loss
              metrics=['accuracy'])


In [ ]:
model.fit(
    train_generator,
    epochs=10,  # Adjust based on your dataset
    steps_per_epoch=train_generator.samples // train_generator.batch_size
)

In [ ]:
for layer in base_model.layers[-30:]:  # Unfreeze the last 30 layers
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=5,
    steps_per_epoch=train_generator.samples // train_generator.batch_size
)


In [ ]:
# Make predictions on the unknown images
predictions = model.predict(unknown_generator, steps=unknown_generator.samples // unknown_generator.batch_size)

# Access the filenames of the unknown images
filenames = unknown_generator.filenames

# Convert predictions to class labels (0 or 1)
predicted_classes = [1 if pred > 0.5 else 0 for pred in predictions]

# Output the results
for filename, pred_class in zip(filenames, predicted_classes):
    label = 'cluster' if pred_class == 1 else 'not cluster'
    print(f"{filename}: {label}")
